In [1]:
import os
import re
import requests
import pandas as pd
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
HTML_PATH = './html'
if not os.path.exists(HTML_PATH):
    os.makedirs(HTML_PATH)

In [3]:
#df = pd.read_csv('url.txt', header=None, names=['url'], nrows=5000)
df = pd.read_csv('url.txt', header=None, names=['url'])
df

,url
0,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...
1,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...
2,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...
3,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...
4,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...
...,...
101390,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...
101391,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...
101392,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...
101393,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...


In [4]:
def scrape_rc(r):
    html_fn = os.path.join(HTML_PATH, '{}.html'.format(r.name))
    rc_id = re.findall('src_no=(\d+)', r.url)[0]
    data = {'url': r.url, 'html': html_fn,
            'rc_id': rc_id, 'rc_nf': 1, 'trans_nf': 1,
            'district': None, 'block': None, 'fpsid': None, 'scheme': None, 'nunits': None,
            'ent_wheat': None, 'ent_rice': None, 'ent_pmgkay_wheat': None, 'ent_pmgkay_rice': None,
            'slno': None, 'member': None, 'avail_fps': None, 'month': None, 'year': None,
            'avail_date': None, 'avail_type': None, 'avail_wheat': None, 'avail_rice': None,
            'avail_pmgkay_wheat_old': None, 'avail_pmgkay_rice_old': None,
            'avail_pmgkay_whole_chana': None, 'avail_pmgkay_pulse': None,
            'avail_pmgkay_wheat': None, 'avail_pmgkay_rice': None}
    try:
        html_fn = os.path.join(HTML_PATH, '{}.html'.format(r.name))
        if os.path.exists(html_fn) and (os.path.getsize(html_fn) > 0):
            with open(html_fn) as f:
                txt = f.read()
        else:
            txt = requests.get(r.url, timeout=10).text
            with open(html_fn, 'w') as f:
                f.write(txt)
        dfs = pd.read_html(txt)
        nt = len(dfs)
        data['ntables'] = nt
        if nt > 0 and dfs[0].columns[0][1].startswith('District'):
            data['rc_nf'] = 0
            s = dfs[0].columns[0][1]
            data['district'] = re.findall('District :\s(.*?)\sTaluk', s)[0]
            data['block'] = re.findall('Taluk :\s(.*?)\sFPS', s)[0]
            data['fpsid'] = re.findall('FPS Id :\s(.*?)\sScheme', s)[0]
            data['scheme'] = re.findall('Scheme :\s(.*?)\sNo of Units', s)[0]
            data['nunits'] = re.findall('No of Units :\s(.*?)$', s)[0]
        if nt > 1 and dfs[1].iloc[0][0].startswith('Entitlement'):
                data['ent_wheat'] = dfs[1].iloc[2][0]
                data['ent_rice'] = dfs[1].iloc[2][1]
                data['ent_pmgkay_wheat'] = dfs[1].iloc[2][2]
                data['ent_pmgkay_rice'] = dfs[1].iloc[2][3]
        if nt > 2 and dfs[2].iloc[0][0].startswith('Transaction'):
                data['trans_nf'] = 0
                data['slno'] = dfs[2].iloc[3][0]
                data['member'] = dfs[2].iloc[3][1]
                data['avail_fps'] = dfs[2].iloc[3][2]
                data['month'] = dfs[2].iloc[3][3]
                data['year'] = dfs[2].iloc[3][4]
                data['avail_date'] = dfs[2].iloc[3][5]
                data['avail_type'] = dfs[2].iloc[3][6]
                data['avail_wheat'] = dfs[2].iloc[3][7]
                data['avail_rice'] = dfs[2].iloc[3][8]
                data['avail_pmgkay_wheat_old'] = dfs[2].iloc[3][9]
                data['avail_pmgkay_rice_old'] = dfs[2].iloc[3][10]
                data['avail_pmgkay_whole_chana'] = dfs[2].iloc[3][11]
                data['avail_pmgkay_pulse'] = dfs[2].iloc[3][12]
                data['avail_pmgkay_wheat'] = dfs[2].iloc[3][13]
                data['avail_pmgkay_rice'] = dfs[2].iloc[3][14]
        if nt > 3 and dfs[3].iloc[0][0].startswith('Transaction'):
                data['trans_nf'] = 0
                data['slno'] = dfs[3].iloc[3][0]
                data['member'] = dfs[3].iloc[3][1]
                data['avail_fps'] = dfs[3].iloc[3][2]
                data['month'] = dfs[3].iloc[3][3]
                data['year'] = dfs[3].iloc[3][4]
                data['avail_date'] = dfs[3].iloc[3][5]
                data['avail_type'] = dfs[3].iloc[3][6]
                data['avail_wheat'] = dfs[3].iloc[3][7]
                data['avail_rice'] = dfs[3].iloc[3][8]
                data['avail_pmgkay_wheat_old'] = dfs[3].iloc[3][9]
                data['avail_pmgkay_rice_old'] = dfs[3].iloc[3][10]
                data['avail_pmgkay_whole_chana'] = dfs[3].iloc[3][11]
                data['avail_pmgkay_pulse'] = dfs[3].iloc[3][12]
                data['avail_pmgkay_wheat'] = dfs[3].iloc[3][13]
                data['avail_pmgkay_rice'] = dfs[3].iloc[3][14]
    except Exception as e:
        print(e, r.name, r.url)
        data['ntables'] = 0
    return pd.Series(data)

In [5]:
%%time
odf = df.parallel_apply(scrape_rc, axis=1)
odf

No tables found 56 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10010070059019700112&month=11&year=2021
No tables found 88721 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10320140023003800017&month=11&year=2021
No tables found 141 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102030101921682100000450&month=11&year=2021
No tables found 146 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102030101921682600001342&month=11&year=2021
No tables found 150 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102030101921682100000437&month=11&year=2021
No tables found 184 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102030101921689000000422&month=11&year=2021
No tables found 189 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10010070056017820009&month=11&year=2021
No tables found 192 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102030101921682800000901&month=11&year=2021
No tables found 25352 http://epos.bihar.gov.in/SRC_Trans_Details.js

No tables found 1233 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10010070051017000062&month=11&year=2021
No tables found 1295 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10010070056017600126&month=11&year=2021
No tables found 1343 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10010070054003510005&month=11&year=2021
No tables found 38045 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10130090170050200011&month=11&year=2021
No tables found 38047 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10130090170051500092&month=11&year=2021
Document is empty 38048 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102150118922739600002105&month=11&year=2021
No tables found 1487 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10010070054003600067&month=11&year=2021
No tables found 1490 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10010070054003510078&month=11&year=2021
No tables found 1543 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src

No tables found 3037 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10020270064015800105&month=11&year=2021
No tables found 3055 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10020270064015800104&month=11&year=2021
No tables found 88770 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102340147025218000000104&month=11&year=2021
No tables found 3114 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10020270035007620116&month=11&year=2021
No tables found 3119 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10020270035007620081&month=11&year=2021
No tables found 3146 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10020270035007620100&month=11&year=2021
No tables found 3148 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10020270035007620053&month=11&year=2021
No tables found 3170 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102040105721866300000313&month=11&year=2021
No tables found 3180 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src

No tables found 5005 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10020270039008900064&month=11&year=2021
No tables found 50784 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10170200031012000022&month=11&year=2021
No tables found 63464 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102240133323891100002308&month=11&year=2021
No tables found 25445 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10090070049014700119&month=11&year=2021
No tables found 12775 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10040110011004010037&month=11&year=2021
No tables found 12776 No tables foundhttp://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10040110011004200103&month=11&year=2021 
88821 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10320140022003300071&month=11&year=2021
No tables found 5023 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10020270026006400065&month=11&year=2021
No tables found 12778 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?s

No tables found 5272 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10020270055014220007&month=11&year=2021
No tables found 89071 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10320140196035900160&month=11&year=2021
No tables found 89076 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102340147025223700000405&month=11&year=2021
No tables found 5281 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10020270055014210100&month=11&year=2021
No tables found 25717 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10090070052018700001&month=11&year=2021
No tables found 89086 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102340147025223700000400&month=11&year=2021
No tables found 76415 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102290138824464300000002&month=11&year=2021
No tables found 51073 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102190126723444300001029&month=11&year=2021
No tables found 13042 http://epos.bihar.gov.in/SRC_Trans_D

...

No tables found 62600 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10220050002002500039&month=11&year=2021
No tables found 75297 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102290138824463100000162&month=11&year=2021
No tables found 100642 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102400153925951600000521&month=11&year=2021
No tables found 62615 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102240133323890800002925&month=11&year=2021
No tables found 49924 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10160080039018500032&month=11&year=2021
No tables found 75309 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102290138824463100000163&month=11&year=2021
No tables found 37263 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102150118922737000000530&month=11&year=2021
No tables found 37266 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102150118922738400000885&month=11&year=2021
No tables found 100657 http://epos.bihar.gov.in

No tables found 75526 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102290138824465100000446&month=11&year=2021
No tables found 24774 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102110113422341400000303&month=11&year=2021
No tables found 24775 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10090070019006700036&month=11&year=2021
No tables found 100878 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10360030080025610109&month=11&year=2021
No tables found 37488 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102150118922737700000723&month=11&year=2021
No tables found 37495 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102150118922735800000778&month=11&year=2021
No tables found 100891 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10360030080025610011&month=11&year=2021
No tables found 75545 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102290138824465100000369&month=11&year=2021
No tables found 75548 http://epos.bihar.gov.in/SRC

No tables found 88434 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102340147025219800000731&month=11&year=2021
No tables found 101120 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102400153925949200000095&month=11&year=2021
No tables found 50385 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102190126723448500000602&month=11&year=2021
No tables found 50387 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102190126723448500000545&month=11&year=2021
No tables found 37734 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10130098012000800102&month=11&year=2021
No tables found 63099 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10220050002002010060&month=11&year=2021
No tables found 37738 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10130098012000400017&month=11&year=2021
No tables found 37741 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10130098012000800038&month=11&year=2021
HTTPConnectionPool(host='epos.bihar.gov.in', port=80): 

No tables found 88560 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102340147025219800000724&month=11&year=2021
No tables found 75910 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102290138824464800000199&month=11&year=2021
No tables found 50513 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102190126723446500000184&month=11&year=2021
No tables found 75935 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102290138824464800000182&month=11&year=2021
No tables found 37871 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10130098012000800109&month=11&year=2021
No tables found 25132 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102110113422341400000327&month=11&year=2021
No tables found 50523 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10170200071024200059&month=11&year=2021
No tables found 75938 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102290138824464500000474&month=11&year=2021
No tables found 75943 http://epos.bihar.gov.in/S

,url,html,rc_id,rc_nf,trans_nf,district,block,fpsid,scheme,nunits,...,avail_type,avail_wheat,avail_rice,avail_pmgkay_wheat_old,avail_pmgkay_rice_old,avail_pmgkay_whole_chana,avail_pmgkay_pulse,avail_pmgkay_wheat,avail_pmgkay_rice,ntables
0,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...,./html/0.html,10010070059019900049,0,0,Pashchim Champaran,Bagaha,120300200239,PHH,7,...,Bio,14.000,21.000,0.000,0.000,0.000,0.000,14.000,21.000,4
1,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...,./html/1.html,10010070059019900064,0,0,Pashchim Champaran,Bagaha,120300200239,PHH,4,...,Iris,8.000,12.000,0.000,0.000,0.000,0.000,8.000,12.000,4
2,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...,./html/2.html,10010070059019900081,0,0,Pashchim Champaran,Bagaha,120300200239,PHH,8,...,Bio,16.000,24.000,0.000,0.000,0.000,0.000,16.000,24.000,4
3,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...,./html/3.html,10010070059019700045,0,0,Pashchim Champaran,Bagaha,120300200243,PHH,5,...,Iris,10.000,15.000,0.000,0.000,0.000,0.000,10.000,15.000,4
4,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...,./html/4.html,10010070059019700073,0,0,Pashchim Champaran,Bagaha,120300200243,PHH,5,...,Bio,10.000,15.000,0.000,0.000,0.000,0.000,10.000,15.000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101390,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...,./html/101390.html,102400153925949800000819,0,1,Jamui,Jamui,124000101125,PHH,2,...,None,None,None,None,None,None,None,None,None,2
101391,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...,./html/101391.html,102400153925951600000539,0,0,Jamui,Jamui,124000101125,PHH,3,...,Bio,6.000,9.000,0.000,0.000,0.000,0.000,6.000,9.000,4
101392,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...,./html/101392.html,102400153925949800000822,0,0,Jamui,Jamui,124000101125,PHH,2,...,Bio,4.000,6.000,0.000,0.000,0.000,0.000,4.000,6.000,4
101393,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...,./html/101393.html,102400153925949700000500,0,0,Jamui,Jamui,124000101125,PHH,1,...,Bio,2.000,3.000,0.000,0.000,0.000,0.000,2.000,3.000,4


In [7]:
odf.to_csv('output_all_r1.csv', index=False)

## Re-run again to fix some zero html file size.

In [8]:
%%time
odf = df.parallel_apply(scrape_rc, axis=1)
odf

No tables found 25352 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102110113422345900000545&month=11&year=2021
No tables found 88721 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10320140023003800017&month=11&year=2021No tables found
 25353 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102110113422345900000464&month=11&year=2021
No tables found 38027 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10130090170050200089&month=11&year=2021
No tables found 76058 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102290138824464500000477&month=11&year=2021
No tables found 25362 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102110113422345900000481&month=11&year=2021
No tables found 25364 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102110113422346200000021&month=11&year=2021
No tables found 25365 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102110113422346200000024&month=11&year=2021
No tables found 25368 http://epos.bihar.gov.in/S

No tables found 25598 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10090070049014700092&month=11&year=2021
No tables found 251 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102030101921682100000456&month=11&year=2021
No tables found 252 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102030101921682100000451&month=11&year=2021
No tables foundNo tables found  254 88939http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102030101921682100000446&month=11&year=2021 
http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10320140022003300046&month=11&year=2021No tables found
 76293 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102290138824464000000714&month=11&year=2021
No tables found 63669 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10220050002001420191&month=11&year=2021
No tables found 12955 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10040110011003910020&month=11&year=2021
No tables found 38095 http://epos.bihar.gov.in/SRC_Trans_Detai

No tables found 89146 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102340147025223700000397&month=11&year=2021
No tables found 38338 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102150118922733000000724&month=11&year=2021
No tables found 38341 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102150118922732200000668&month=11&year=2021
No tables found 89151 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102340147025223700000408&month=11&year=2021
No tables found 76579 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102290138824465500000113&month=11&year=2021
No tables found 89161 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102340147025223500000464&month=11&year=2021
No tables found 89163 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102340147025223500000469&month=11&year=2021
No tables found 63967 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102240133323890500000355&month=11&year=2021
No tables found 51184 http://epos.bihar.

No tables found 13395 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102060107321945700000190&month=11&year=2021
No tables found 76845 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102290138824463300000677&month=11&year=2021
No tables found 26052 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102110113422344000001181&month=11&year=2021
No tables found 26054 No tables foundhttp://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10090070040012820035&month=11&year=2021 
76856 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102290138824463300000395&month=11&year=2021No tables found
 786 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102030101921677300000430&month=11&year=2021
No tables found 789 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102030101921677200000284&month=11&year=2021
No tables found 89418 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10320140089015800042&month=11&year=2021
No tables found 51437 http://epos.bihar.gov.in/SRC_T

No tables found 64321 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102240133323891200004417&month=11&year=2021
No tables found 64322No tables found http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102240133323891200004389&month=11&year=2021
 936 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10010070063001900064&month=11&year=2021
No tables found 89569 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10320140089015800141&month=11&year=2021
No tables found 943 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102030101921677600000237&month=11&year=2021
No tables found 76983 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102290138824465700000469&month=11&year=2021
No tables found No tables found No tables found8957838689   http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102340147025215100000028&month=11&year=2021
http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102150118922734200000349&month=11&year=202126207
No tables found  http://epos.bi

...

No tables found No tables found11964  88189 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10320140168030900059&month=11&year=2021http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102060107321942300000006&month=11&year=2021

No tables found 49213 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10160080016007900058&month=11&year=2021
No tables found 75297 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102290138824463100000162&month=11&year=2021
No tables found 11974 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102060107321940800000303&month=11&year=2021
No tables found No tables found11975  http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10040110038017700028&month=11&year=202162802
 No tables found http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10220050004004800052&month=11&year=202111976
 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10040110013019000109&month=11&year=2021
No tables found 75309 http://epos.bihar.gov.in/SRC_Trans_Det

No tables found 25143 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10090070021007100044&month=11&year=2021
No tables found 49454 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10160080016007300015&month=11&year=2021
No tables found 63074 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102240133323890600002681&month=11&year=2021
No tables found 88388 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102340147025216400000030&month=11&year=2021
No tables found 49462 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10160080016007200102&month=11&year=2021
No tables found 88395 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10320140168030800022&month=11&year=2021
No tables found 63088 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10220050002002120030&month=11&year=2021
No tables found 49478 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10160080016007500007&month=11&year=2021
No tables found 12258 http://epos.bihar.gov.in/SRC_Trans_Details

No tables found 100796 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10360030075023900008&month=11&year=2021
No tables found 49663 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10160080016007400018&month=11&year=2021
No tables found 49664 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102180123623177400001036&month=11&year=2021
No tables found 25313 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102110113422346200000006&month=11&year=2021
No tables found 75874 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102290138824464800000265&month=11&year=2021
No tables found 12440 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10040110038017300219&month=11&year=2021
No tables found 100816 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10360030075024000072&month=11&year=2021
No tables found 88560 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102340147025219800000724&month=11&year=2021No tables found
 100819 http://epos.bihar.gov.in/SRC_Tr

No tables found 101079 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102400153925949100000392&month=11&year=2021
No tables found 101096 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102400153925949100000418&month=11&year=2021
No tables found 49924 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10160080039018500032&month=11&year=2021
No tables found 101103 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102400153925949100000399&month=11&year=2021
No tables found 101110No tables found  http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102400153925949100000422&month=11&year=202149933
 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10160080038017909999&month=11&year=2021
No tables found 101120 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=102400153925949200000095&month=11&year=2021
No tables found 49967 http://epos.bihar.gov.in/SRC_Trans_Details.jsp?src_no=10160080038017900072&month=11&year=2021
No tables found 101142 http://epos.bihar.gov.in

,url,html,rc_id,rc_nf,trans_nf,district,block,fpsid,scheme,nunits,...,avail_type,avail_wheat,avail_rice,avail_pmgkay_wheat_old,avail_pmgkay_rice_old,avail_pmgkay_whole_chana,avail_pmgkay_pulse,avail_pmgkay_wheat,avail_pmgkay_rice,ntables
0,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...,./html/0.html,10010070059019900049,0,0,Pashchim Champaran,Bagaha,120300200239,PHH,7,...,Bio,14.000,21.000,0.000,0.000,0.000,0.000,14.000,21.000,4
1,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...,./html/1.html,10010070059019900064,0,0,Pashchim Champaran,Bagaha,120300200239,PHH,4,...,Iris,8.000,12.000,0.000,0.000,0.000,0.000,8.000,12.000,4
2,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...,./html/2.html,10010070059019900081,0,0,Pashchim Champaran,Bagaha,120300200239,PHH,8,...,Bio,16.000,24.000,0.000,0.000,0.000,0.000,16.000,24.000,4
3,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...,./html/3.html,10010070059019700045,0,0,Pashchim Champaran,Bagaha,120300200243,PHH,5,...,Iris,10.000,15.000,0.000,0.000,0.000,0.000,10.000,15.000,4
4,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...,./html/4.html,10010070059019700073,0,0,Pashchim Champaran,Bagaha,120300200243,PHH,5,...,Bio,10.000,15.000,0.000,0.000,0.000,0.000,10.000,15.000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101390,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...,./html/101390.html,102400153925949800000819,0,1,Jamui,Jamui,124000101125,PHH,2,...,None,None,None,None,None,None,None,None,None,2
101391,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...,./html/101391.html,102400153925951600000539,0,0,Jamui,Jamui,124000101125,PHH,3,...,Bio,6.000,9.000,0.000,0.000,0.000,0.000,6.000,9.000,4
101392,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...,./html/101392.html,102400153925949800000822,0,0,Jamui,Jamui,124000101125,PHH,2,...,Bio,4.000,6.000,0.000,0.000,0.000,0.000,4.000,6.000,4
101393,http://epos.bihar.gov.in/SRC_Trans_Details.jsp...,./html/101393.html,102400153925949700000500,0,0,Jamui,Jamui,124000101125,PHH,1,...,Bio,2.000,3.000,0.000,0.000,0.000,0.000,2.000,3.000,4


In [9]:
odf.to_csv('output_all_r2.csv', index=False)